In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [64]:
!wc -l ./itineraries_random_2M.csv

484552 ./itineraries_random_2M.csv


In [4]:
df_raw = pd.read_csv('./itineraries_random_2M.csv')
df_raw

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,620cd51404373b9bdbbc46a0b657573c,2022-05-29,2022-06-18,LAX,ATL,Y0N,PT4H19M,0,False,False,...,1655606940,2022-06-18T22:49:00.000-04:00,ATL,LAX,Delta,DL,Airbus A321,15540,1943,coach
1,8622d132e890b398c0ad1d62b0232f54,2022-05-03,2022-06-24,CLT,LAX,MA0OA0MQ,PT8H10M,0,False,False,...,1656078240||1656103200,2022-06-24T09:44:00.000-04:00||2022-06-24T13:4...,ATL||LAX,CLT||ATL,Delta||Delta,DL||DL,Boeing 717||Airbus A321,4440||16680,228||1943,coach||coach
2,6f419788769ddb3802d047272ad0a54d,2022-05-02,2022-06-30,LGA,ATL,LAVQA0ML,PT11H54M,1,False,False,...,1656645360||1656676380,2022-06-30T23:16:00.000-04:00||2022-07-01T07:5...,MIA||ATL,LGA||MIA,Delta||Delta,DL||DL,Boeing 737-800||Boeing 737-900,11820||6780,1104||596,coach||coach
3,41f3cd0a63603895e13b5b77630bd84e,2022-06-10,2022-08-04,LAX,SFO,MH7OASMN,PT1H20M,0,False,False,...,1659669600,2022-08-04T20:20:00.000-07:00,SFO,LAX,Alaska Airlines,AS,Embraer 175,4800,339,coach
4,7f5d8684b672dd3593e30ee946264c31,2022-06-24,2022-07-20,LGA,DFW,LAVQA0BQ,PT6H41M,0,True,False,...,1658337240||1658351760,2022-07-20T13:14:00.000-04:00||2022-07-20T16:1...,ATL||DFW,LGA||ATL,Delta||Delta,DL||DL,Airbus A320||Airbus A321,9540||8460,762||725,coach||coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14107,9540a92466a02a03e4e8b26811f5cb2c,2022-06-01,2022-06-20,ORD,BOS,QAA0OHEN,PT2H24M,0,False,False,...,1655741820,2022-06-20T12:17:00.000-04:00,BOS,ORD,United,UA,Airbus A320,8640,862,coach
14108,6386e96f3ddb65a0d8dac8aa7b873928,2022-06-04,2022-07-28,LGA,PHL,GUAHZNN1,PT4H26M,0,False,False,...,1659056820||1659065100,2022-07-28T21:07:00.000-04:00||2022-07-28T23:2...,CLT||PHL,LGA||CLT,American Airlines||American Airlines,AA||AA,Boeing 737-800||Airbus A321,7680||5640,545||449,coach||coach
14109,a3d9fde3055933425de397889f6b2dc0,2022-04-30,2022-05-14,EWR,DTW,TAUSA5BC,PT1H47M,0,True,False,...,1652554020,2022-05-14T14:47:00.000-04:00,DTW,EWR,Delta,DL,Airbus A220-100,6420,485,coach
14110,1fb66e26337157d6b606bc3b2ed2e089,2022-04-19,2022-05-12,LGA,ATL,ZI4AUEL1,PT8H23M,1,False,False,...,1652397840||1652415780,2022-05-12T19:24:00.000-04:00||2022-05-13T00:2...,FLL||ATL,LGA||FLL,JetBlue Airways||JetBlue Airways,B6||B6,Airbus A320||Airbus A320,12240||6720,1085||582,coach||coach


In [5]:
# how many premium?
is_premium = df_raw['segmentsCabinCode'].str.contains('business') | df_raw['segmentsCabinCode'].str.contains('premium') | df_raw['segmentsCabinCode'].str.contains('first')
len(df_raw[is_premium]) / len(df_raw)

0.004464285714285714

In [6]:
df_raw['isRefundable'].sum()

0

In [38]:
df_raw.isna().sum()

,0
legId,0
searchDate,0
flightDate,0
startingAirport,0
destinationAirport,0
fareBasisCode,0
travelDuration,0
elapsedDays,0
isBasicEconomy,0
isRefundable,0


In [39]:
df = df_raw[~is_premium & ~df_raw['isRefundable']].copy()
df = df.drop(['segmentsCabinCode', 'isRefundable'], axis=1)
df = df.dropna()

In [40]:
# drop duplicate cols
df = df.drop(['segmentsAirlineName', 'segmentsDepartureTimeRaw', 'segmentsArrivalTimeRaw', 'isBasicEconomy', 'isNonStop'], axis=1)
# drop irrelevans cols
df = df.drop([
    'legId',
    'fareBasisCode',
    'searchDate',
    'seatsRemaining',
    'baseFare',
    'segmentsEquipmentDescription'
], axis=1)

In [41]:
def parse_duration(i):
  if not i[2]:
    return int(i[1])
  return 60 * int(i[1]) + int(i[2])
df['travelDuration'] = df['travelDuration'].str.split(r'[^0-9]+').map(parse_duration)

In [42]:
df['stops'] = df['segmentsArrivalAirportCode'].str.count(r'\|\|')

In [43]:
# prepare dates
dates = pd.to_datetime(df['flightDate'])
df['day_of_week'] = dates.dt.day_of_week
df['month'] = dates.dt.month
df = df.drop('flightDate', axis=1)

In [44]:
df_bare = df.drop([
    'segmentsDepartureTimeEpochSeconds',
    'segmentsArrivalTimeEpochSeconds',
    'segmentsArrivalAirportCode',
    'segmentsDepartureAirportCode',
    'segmentsAirlineCode',
    'segmentsDurationInSeconds',
    'segmentsDistance',
    # 'travelDuration',
    # 'totalTravelDistance'
], axis=1)

In [45]:
df_bare

,startingAirport,destinationAirport,travelDuration,elapsedDays,totalFare,totalTravelDistance,stops,day_of_week,month
0,LAX,ATL,259,0,1028.61,1943.0,0,5,6
1,CLT,LAX,490,0,537.10,2171.0,1,4,6
2,LGA,ATL,714,1,382.21,1700.0,1,3,6
3,LAX,SFO,80,0,218.60,339.0,0,3,8
4,LGA,DFW,401,0,207.60,1487.0,1,2,7
...,...,...,...,...,...,...,...,...,...
14107,ORD,BOS,144,0,288.60,862.0,0,0,6
14108,LGA,PHL,266,0,371.11,994.0,1,3,7
14109,EWR,DTW,107,0,128.60,485.0,0,5,5
14110,LGA,ATL,503,1,170.61,1667.0,1,3,5


# Numerify

In [46]:
df_num = df_bare
categorical_columns = ['startingAirport', 'destinationAirport']
for col in categorical_columns:
  col_ohe = pd.get_dummies(df_bare[col], prefix=col).astype(int)
  df_num = pd.concat((df_num, col_ohe), axis=1).drop(col, axis=1)

In [47]:
X = df_num.drop('totalFare', axis=1)
y = df_num['totalFare']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,travelDuration,elapsedDays,totalTravelDistance,stops,day_of_week,month,startingAirport_ATL,startingAirport_BOS,startingAirport_CLT,startingAirport_DEN,...,destinationAirport_EWR,destinationAirport_IAD,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_LGA,destinationAirport_MIA,destinationAirport_OAK,destinationAirport_ORD,destinationAirport_PHL,destinationAirport_SFO
8738,157,0,720.0,0,6,8,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5941,432,1,2463.0,1,1,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7657,366,0,990.0,1,1,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
837,86,0,185.0,0,6,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5804,280,0,725.0,1,0,5,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9557,626,1,2751.0,1,4,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10756,320,0,953.0,1,5,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7816,619,0,1876.0,1,1,6,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
12848,365,0,2458.0,0,4,5,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## baseline: linear regression

In [48]:
from sklearn.linear_model import LinearRegression

In [49]:
lr_model = LinearRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
display(f'R2: {r2_score(y_test, y_pred)}')
display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

'R2: 0.516609206438495'

'MAPE: 0.32686201569207873'

## random forest

In [51]:
from sklearn.ensemble import RandomForestRegressor

In [52]:
rf_model = RandomForestRegressor().fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
display(f'R2: {r2_score(y_test, y_pred)}')
display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

'R2: 0.568774030306918'

'MAPE: 0.2866239116423989'

In [ ]:
params = { 'max_depth': [3,10,15,None], 'n_estimators': [50, 100, 200] }
hp_model = GridSearchCV(RandomForestRegressor(), params).fit(X_train, y_train)
print(hp_model.best_params_)

## catboost

In [54]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


In [55]:
from catboost import CatBoostRegressor

In [56]:
cb_model = CatBoostRegressor().fit(X_train, y_train)
y_pred = cb_model.predict(X_test)
display(f'R2: {r2_score(y_test, y_pred)}')
display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

Learning rate set to 0.059078
0:	learn: 193.6229320	total: 65.5ms	remaining: 1m 5s
1:	learn: 188.9389755	total: 78.7ms	remaining: 39.3s
2:	learn: 184.6137912	total: 111ms	remaining: 36.9s
3:	learn: 180.5696805	total: 119ms	remaining: 29.7s
4:	learn: 177.1033819	total: 129ms	remaining: 25.7s
5:	learn: 173.7458308	total: 138ms	remaining: 22.9s
6:	learn: 170.6599183	total: 142ms	remaining: 20.2s
7:	learn: 167.9721103	total: 153ms	remaining: 19s
8:	learn: 165.5434625	total: 163ms	remaining: 18s
9:	learn: 163.1435568	total: 170ms	remaining: 16.8s
10:	learn: 160.9729907	total: 184ms	remaining: 16.6s
11:	learn: 159.0434481	total: 192ms	remaining: 15.8s
12:	learn: 157.2969409	total: 199ms	remaining: 15.1s
13:	learn: 155.5910029	total: 208ms	remaining: 14.6s
14:	learn: 154.1050530	total: 215ms	remaining: 14.1s
15:	learn: 152.7719677	total: 224ms	remaining: 13.8s
16:	learn: 151.5286748	total: 233ms	remaining: 13.5s
17:	learn: 150.3125504	total: 242ms	remaining: 13.2s
18:	learn: 149.2114142	total

'R2: 0.6155586266121533'

'MAPE: 0.27198875884824164'